### Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
%pip install semantic-link-labs

### Import the library and obtain a TokenProvider using Azure Key Vault

In [ ]:
import sempy_labs as labs
from sempy_labs import admin

token_provider = labs.ServicePrincipalTokenProvider.from_azure_key_vault(
    key_vault_uri = '', # Enter your key vault URI
    key_vault_tenant_id = '', # Enter the key vault secret storig your Tenant ID
    key_vault_client_id = '', # Enter the key vault secret storig your Client ID (Applciation ID)
    key_vault_client_secret = '' # Enter the key vault secret storing your Client Secret
)

### Use the generated TokenProvider to authenticate in other functions

In [ ]:
admin.list_capacities(token_provider=token_provider)